# Imports

In [ ]:
import pymongo
import os
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn_image as sns_img
from random import choice

# Connect to database

In [ ]:
try:
    database_url = "mongodb://localhost:27017/"
    client = pymongo.MongoClient(database_url)
    client.server_info()
except pymongo.errors.ServerSelectionTimeoutError as err:
    raise ConnectionError

database = client["MNIST"]
std_dataset = database["dataset"]
meta_collection = database["metamorphical"]

# Get a picture

In [ ]:
def get_picture(query = {}):
    pipeline = [
    {
        "$sample": {
            "size": 1  # Get 1 random document
        }
    }
    ]
    pick = std_dataset.aggregate(pipeline)
    pick = pick.to_list()[0]
    image = numpy.reshape(pick["image"], (28, 28))
    return image, pick["label"]

def get_meta_picture(query = {}):
    pipeline = [
    {
        "$match": query
    },
    {
        "$sample": {
            "size": 1  # Get 1 random document
        }
    }
    ]
    pick = meta_collection.aggregate(pipeline)
    pick = pick.to_list()[0]
    image = numpy.reshape(pick["image"], (28, 28))
    pick["image"] = image
    return pick


In [ ]:

a_pic, a_label = get_picture()
print(f"Label: {a_label}")
plt.imshow(a_pic, cmap=sns.color_palette("Blues", as_cmap=True))


In [ ]:

meta_pic = get_meta_picture({"status":"noise"})
print(f"Label: {meta_pic["label"]} | Status: {meta_pic["status"]}")
plt.imshow(meta_pic["image"], cmap=sns.color_palette("Blues", as_cmap=True))

# Transform a picture

In [ ]:
# Get 5000 data points

data_points = std_dataset.aggregate([{"$sample": {"size": 5000}}])
x = []
y = []
for item in data_points:
    x.append(item["image"])
    y.append(item["label"])

# Visualize based on the 2 componentes and the label.

# 


In [ ]:
from sklearn.decomposition import PCA, SparsePCA

pca = PCA(n_components=2)
s_pca = SparsePCA(n_components=2)

points = s_pca.fit_transform(x)

In [ ]:
points
x_vals, y_vals = zip(*points)

# Create a scatter plot
plt.scatter(x_vals, y_vals, c=y, cmap='viridis', s=100)

In [ ]:

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
np_x = numpy.array(x)
x_tsne = tsne.fit_transform(np_x)


In [ ]:
x_tsne
x_vals, y_vals = zip(*x_tsne)

# Create a scatter plot
plt.scatter(x_vals, y_vals, c=y, cmap='viridis', s=100)

In [ ]:
from matplotlib.colors import BoundaryNorm

bound_norm = BoundaryNorm(range(0,11), 10)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(x_vals, y_vals, c=y, cmap='tab10', s=100)
# Add color bar to show the mapping of numbers to colors
plt.colorbar(scatter, label='Color Value')